# LinkdIn Jobs Scrapper

## Imports

In [1]:
from selenium import webdriver
import time
import pandas as pd
import os

from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

import re
from bs4 import BeautifulSoup

import numpy as np

In [2]:
linkdInJobsUrl = "https://www.linkedin.com/jobs/search?keywords=Microsoft%20Power%20Platform&location=India&geoId=102713980&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0"

## WebDriver

In [3]:
#Creating driver instance
driver1 = webdriver.Chrome(executable_path=r'".\\chromedriver.exe"')

#fetching the data
driver1.get(linkdInJobsUrl)
driver1.implicitly_wait(15)

C:\Users\vibhu\AppData\Local\Temp\ipykernel_7124\84967656.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(executable_path=r'".\\chromedriver.exe"')
Incompatible release of chromedriver (version 114.0.5735.16) detected in PATH: D:\Coding\projectsForResume\PowerBI\linkdIn_analyzer\linkdin_scrapper\chromedriver.exe


### Finding the total results

In [4]:
jobResultCountObj = driver1.find_element(By.CLASS_NAME,'results-context-header__job-count')
jobResultCountHtml = jobResultCountObj.get_attribute("outerHTML")

In [5]:
jobResultCountHtml

'<span class="results-context-header__job-count">1,000+</span>'

In [6]:
soup1 = BeautifulSoup(jobResultCountHtml,"html.parser")

In [7]:
soup1.text

'1,000+'

In [8]:
jobResultCountText = ""
for i in soup1.text:
    if i >= '0' and i <= '9' :
        jobResultCountText += i

jobResultCount = pd.to_numeric(jobResultCountText)

In [9]:
jobResultCount

1000

### Calculating the loops needed
#### Assumption : 20 items are being loaded in single scroll

In [10]:
loops = 200

## Making sure that driver keeps scrolling to the bottom of the page and loads new stuff

In [11]:
i=1
while i<=loops:
    #Scroll to bottom
    driver1.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    i+=1
    
    try:
        #Press See More Jobs button when encountered
        x=driver1.find_element(By.XPATH,"//button[@aria-label='See more jobs']")
        driver1.execute_script("arguments[0].click();",x)
        time.sleep(5)
    except:
        pass
        time.sleep(4)

## Storing the loaded HTML into a component

In [12]:
loadedDataObj = driver1.find_element(By.TAG_NAME,"body")

In [13]:
loadedDataHTML = loadedDataObj.get_attribute("outerHTML")

In [14]:
loadedDataHTML

'<body dir="ltr">\n<!----><!----><!---->\n        \n          \n    \n\n    <form class="google-auth" action="https://www.linkedin.com/uas/login-submit" method="post">\n      <input name="loginCsrfParam" value="b48a91f0-fbd9-454c-81e3-dbd6b88b9178" type="hidden">\n\n        <input name="session_redirect" value="https://www.linkedin.com/jobs/search?keywords=Microsoft%20Power%20Platform&amp;location=India&amp;geoId=102713980&amp;trk=public_jobs_jobs-search-bar_search-submit&amp;position=1&amp;pageNum=0" type="hidden">\n\n      <input name="trk" value="public_jobs_google-one-tap-submit" type="hidden">\n        <div id="google-one-tap__container" data-tracking-control-name="public_jobs_google-one-tap"></div>\n      \n    <div class="loader loader--full-screen">\n      <div class="loader__container mb-2 overflow-hidden">\n        <icon class="loader__icon inline-block loader__icon--default text-color-progress-loading lazy-loaded" data-svg-class-name="loader__icon-svg--large fill-currentColo

## Using BeautifulSoup to parse the html

In [15]:
soupMain = BeautifulSoup(loadedDataHTML,"html.parser")

## Scooping the information into arrays

In [16]:
companyNames = []
jobTitles = []
companyLocations = []
linkToJobPostings = []


In [17]:
companyNamesHTML = soupMain.find_all("a",class_="hidden-nested-link")
jobTitlesHTML = soupMain.find_all("h3",class_="base-search-card__title")
companyLocationsHTML = soupMain.find_all("span",class_="job-search-card__location")
linkToJobPostingsHTML = soupMain.find_all("a",class_="hidden-nested-link",href=True)

In [18]:
n = len(companyNamesHTML)
for i in range(0,n) :
    companyNames.append(companyNamesHTML[i].text)
    jobTitles.append(jobTitlesHTML[i].text)
    companyLocations.append(companyLocationsHTML[i].text)
    linkToJobPostings.append(linkToJobPostingsHTML[i]["href"])

In [19]:
companyNames,jobTitles,companyLocations,linkToJobPostings

(['\n            CGI\n          ',
  '\n            CodersBrain\n          ',
  '\n            Technovids Consulting Services\n          ',
  '\n            Sapwood Ventures Pvt. Ltd.\n          ',
  '\n            NuStar Technologies\n          ',
  '\n            CodersBrain\n          ',
  '\n            CodersBrain\n          ',
  '\n            Feuji Inc\n          ',
  '\n            ETT CareerMove\n          ',
  '\n            Attrillion Services Private Limited\n          ',
  '\n            Feuji Inc\n          ',
  '\n            Deloitte Asia Pacific\n          ',
  '\n            AgreeYa Solutions\n          ',
  '\n            Sony India Software Centre\n          ',
  '\n            CGI\n          ',
  '\n            CodeValue Technologies Pvt Ltd\n          ',
  '\n            Sony India Software Centre\n          ',
  '\n            Aargee Staffing Services Pvt. Ltd.\n          ',
  '\n            Withum\n          ',
  '\n            Ace Talent Consulting India\n     

## Extracting more informations

In [20]:
jobDescriptions = []
timePosted = []
relativeTime = []
jobId = []
employementDetails=[]

In [21]:
jobListsElements = driver1.find_elements(By.XPATH,"//a[@data-tracking-control-name='public_jobs_jserp-result_search-card']")

In [22]:
n = len(jobListsElements)
driver2 = webdriver.Chrome(executable_path=r'".\\chromedriver.exe"')
for i in range(0,n) :
    pos=""
    pageNum=""
    currentJobId=""

    currElement = jobListsElements[i].get_attribute("outerHTML")
    s2 = BeautifulSoup(currElement,"html.parser")
    link = s2.find("a",href=True)
    
    
    #calculate jobid
    t1=link["href"].find("?refId")
    s = link["href"][:t1]
    k=s.split("-")
    currentJobId=k[len(k)-1]
    
    #for pageNum
    t2 = link["href"].find("&trk")
    s2=link["href"][:t2]
    k2 = s2.split('=')
    pageNum=k2[len(k2)-1]
    
    #position
    t3=link["href"].find("&pageNum")
    s3=link["href"][:t3]
    k3=s3.split("=")
    pos=k3[len(k3)-1]
    
    lindInNewUrl = "https://www.linkedin.com/jobs/search?keywords=Business%2BIntelligence%2B%28BI%29&location=United%2BStates&geoId=103644278&trk=public_jobs_jobs-search-bar_search-submit&currentJobId="+currentJobId+"&position="+pos+"&pageNum="+pageNum
    driver2.get(lindInNewUrl)
    time.sleep(5)
    
    bodyElementObj = driver2.find_element(By.TAG_NAME,"body")
    bodyElementHTML = bodyElementObj.get_attribute("outerHTML")
    soupDetails = BeautifulSoup(bodyElementHTML,"html.parser")
    
    
    
    check = soupDetails.find("h2",class_="top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title")
    while check == None :
        driver2.get(lindInNewUrl)
        time.sleep(5)
    
        bodyElementObj = driver2.find_element(By.TAG_NAME,"body")
        bodyElementHTML = bodyElementObj.get_attribute("outerHTML")
        soupDetails = BeautifulSoup(bodyElementHTML,"html.parser")
        check = soupDetails.find("h2",class_="top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title")
    
    jobId.append(currentJobId)

    jobDescription = soupDetails.find("div",class_="show-more-less-html__markup")
    jobDescriptions.append(jobDescription)

    employementTypeList = soupDetails.find_all("span",class_="description__job-criteria-text description__job-criteria-text--criteria")
    employementDetails.append(employementTypeList)

    relativeTimeHTML1 = soupDetails.find("span",class_="posted-time-ago__text topcard__flavor--metadata")
    relativeTimeHTML2 = soupDetails.find("span",class_="posted-time-ago__text posted-time-ago__text--new topcard__flavor--metadata")
    if relativeTimeHTML1 == None :
        relativeTime.append(relativeTimeHTML2.text)
    else:
        relativeTime.append(relativeTimeHTML1.text)


C:\Users\vibhu\AppData\Local\Temp\ipykernel_7124\2819757070.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver2 = webdriver.Chrome(executable_path=r'".\\chromedriver.exe"')
Incompatible release of chromedriver (version 114.0.5735.16) detected in PATH: D:\Coding\projectsForResume\PowerBI\linkdIn_analyzer\linkdin_scrapper\chromedriver.exe


In [23]:
m=len(companyNames)

In [24]:
jobDescriptions=jobDescriptions[:m]

In [25]:
employementDetails=employementDetails[:m]

In [26]:
relativeTime=relativeTime[:m]

## Creating a DataFrame

In [27]:
df = pd.DataFrame()

In [28]:
df['CompanyNames'] = companyNames

In [29]:
df['JobTitles'] = jobTitles

In [30]:
df['CompanyLocation'] = companyLocations

In [31]:
df['LinkToJobPostings'] = linkToJobPostings

In [32]:
df['JobDescriptions'] = jobDescriptions

In [33]:
employementDetails[7]

[<span class="description__job-criteria-text description__job-criteria-text--criteria">
             Mid-Senior level
           </span>,
 <span class="description__job-criteria-text description__job-criteria-text--criteria">
           Full-time
         </span>,
 <span class="description__job-criteria-text description__job-criteria-text--criteria">
             Design, Art/Creative, and Information Technology
           </span>,
 <span class="description__job-criteria-text description__job-criteria-text--criteria">
           Information Technology &amp; Services
           </span>]

In [34]:
seniorityLevel=[]
employementType=[]
jobFunctions=[]
industries=[]

for i in range(0,len(employementDetails)):
    if len(employementDetails[i]) == 4:
        sp0 = BeautifulSoup(str(employementDetails[i][0]),"html.parser")
        sp1 = BeautifulSoup(str(employementDetails[i][1]),"html.parser")
        sp2 = BeautifulSoup(str(employementDetails[i][2]),"html.parser")
        sp3 = BeautifulSoup(str(employementDetails[i][3]),"html.parser")
        t0 = sp0.text
        t1 = sp1.text
        t2 = sp2.text
        t3 = sp3.text
        seniorityLevel.append(t0)
        employementType.append(t1)
        jobFunctions.append(t2)
        industries.append(t3)
    else :
        seniorityLevel.append("Scrapping Missed")
        employementType.append("Scrapping Missed")
        jobFunctions.append("Scrapping Missed")
        industries.append("Scrapping Missed")

In [35]:
len(seniorityLevel)

936

In [36]:
df['SeniorityLevel'] = seniorityLevel
df['EmployementType'] = employementType
df['JobFunctions'] = jobFunctions
df['Industries'] = industries

In [37]:
df['RelativeTime'] = relativeTime

In [38]:
df['RelativeTime']

0      \n        \n\n    \n    \n    \n    \n    \n  ...
1      \n        \n\n    \n    \n    \n    \n    \n  ...
2      \n        \n\n    \n    \n    \n    \n    \n  ...
3      \n        \n\n    \n    \n    \n    \n    \n  ...
4      \n        \n\n    \n    \n    \n    \n    \n  ...
                             ...                        
931    \n        \n\n    \n    \n    \n    \n    \n  ...
932    \n        \n\n    \n    \n    \n    \n    \n  ...
933    \n        \n\n    \n    \n    \n    \n    \n  ...
934    \n        \n\n    \n    \n    \n    \n    \n  ...
935    \n        \n\n    \n    \n    \n    \n    \n  ...
Name: RelativeTime, Length: 936, dtype: object

In [39]:
df

,CompanyNames,JobTitles,CompanyLocation,LinkToJobPostings,JobDescriptions,SeniorityLevel,EmployementType,JobFunctions,Industries,RelativeTime
0,\n CGI\n,\n \n Power Platform Develop...,"\n Bengaluru, Karnataka, India\n ...",https://ca.linkedin.com/company/cgi?trk=public...,"[\n Position Description, [[]], Externa...",\n Entry level\n,\n Full-time\n,\n Engineering and Information Tech...,\n IT Services and IT Consulting\n ...,\n \n\n \n \n \n \n \n ...
1,\n CodersBrain\n,\n \n POWER PLATFORM DEVELOP...,"\n Bengaluru, Karnataka, India\n ...",https://in.linkedin.com/company/codersbrain?tr...,"[\n Hello,, [[]], Greetings from , [Cod...",\n Associate\n,\n Full-time\n,\n Engineering and Information Tech...,\n IT Services and IT Consulting\n ...,\n \n\n \n \n \n \n \n ...
2,\n Technovids Consulting Services\n...,\n \n Power Apps Developer\n...,"\n Bengaluru North, Karnataka, Indi...",https://in.linkedin.com/company/technovids-con...,"[\n PowerApp Developer, [[]], Responsib...",\n Mid-Senior level\n,\n Full-time\n,\n Engineering and Information Tech...,\n IT Services and IT Consulting\n ...,\n \n\n \n \n \n \n \n ...
3,\n Sapwood Ventures Pvt. Ltd.\n ...,\n \n Power Platform Develop...,"\n Hyderabad, Telangana, India\n ...",https://in.linkedin.com/company/sapwood-ventur...,[\n Job opening in one of the reputed c...,\n Entry level\n,\n Full-time\n,\n Engineering and Information Tech...,"\n Software Development, Technology, ...",\n \n\n \n \n \n \n \n ...
4,\n NuStar Technologies\n,\n \n Power Apps\n \n ...,"\n Hyderabad, Telangana, India\n ...",https://www.linkedin.com/company/nustar-techno...,"[\n, [[Responsibilities]], , [[Manage the Kel...",\n Mid-Senior level\n,\n Full-time\n,\n Other\n,\n Information Technology & Services\...,\n \n\n \n \n \n \n \n ...
...,...,...,...,...,...,...,...,...,...,...
931,\n IBM\n,\n \n Supply Chain Analytics...,"\n Pune, Maharashtra, India\n ...",https://www.linkedin.com/company/ibm?trk=publi...,"[\n R00195113, [[]], Engineering | MCA ...",\n Associate\n,\n Full-time\n,\n Supply Chain\n,\n Chemical Manufacturing\n,\n \n\n \n \n \n \n \n ...
932,\n Cyient\n,\n \n Cloud Data Architect -...,"\n Bengaluru, Karnataka, India\n ...",https://in.linkedin.com/company/cyient?trk=pub...,[\n Urgent requirement for Cloud Data A...,\n Mid-Senior level\n,\n Full-time\n,\n Engineering and Information Tech...,\n Staffing and Recruiting\n,\n \n\n \n \n \n \n \n ...
933,\n Accenture in India\n,\n \n Lead Data Scientist GS...,"\n Bengaluru, Karnataka, India\n ...",https://in.linkedin.com/company/accentureindia...,"[\n Job Description, [], [], The health...",\n Not Applicable\n,\n Full-time\n,\n Engineering and Information Tech...,"\n Banking, Financial Services, and I...",\n \n\n \n \n \n \n \n ...
934,\n Agoda\n,\n \n Microsoft Dynamics 365...,"\n Chennai, Tamil Nadu, India\n ...",https://sg.linkedin.com/company/agoda?trk=publ...,"[\n, [About Accenture: ], Accenture is a globa...",\n Mid-Senior level\n,\n Full-time\n,\n Information Technology and Engin...,\n IT Services and IT Consulting\n ...,\n \n\n \n \n \n \n \n ...


### Cleansing the data

In [40]:
remchar = "\n"

In [41]:
df['CompanyNames'] = df['CompanyNames'].apply(lambda x : x.translate(str.maketrans('','',remchar)))

In [42]:
df['JobTitles'] = df['JobTitles'].apply(lambda x : x.translate(str.maketrans('','',remchar)))

In [43]:
df['CompanyLocation'] = df['CompanyLocation'].apply(lambda x : x.translate(str.maketrans('','',remchar)))

In [44]:
df['SeniorityLevel'] = df['SeniorityLevel'].apply(lambda x : x.translate(str.maketrans('','',remchar)))

In [45]:
df['EmployementType'] = df['EmployementType'].apply(lambda x : x.translate(str.maketrans('','',remchar)))

In [46]:
df['JobFunctions'] = df['JobFunctions'].apply(lambda x : x.translate(str.maketrans('','',remchar)))

In [47]:
df['Industries'] = df['Industries'].apply(lambda x : x.translate(str.maketrans('','',remchar)))

In [48]:
df['RelativeTime'] = df['RelativeTime'].apply(lambda x : x.translate(str.maketrans('','',remchar)))

In [49]:
len(df.iat[0,0])

25

In [50]:
df['CompanyNames'] = df['CompanyNames'].apply(lambda x : x.strip())
df['JobTitles'] = df['JobTitles'].apply(lambda x : x.strip())
df['CompanyLocation'] = df['CompanyLocation'].apply(lambda x : x.strip())
df['SeniorityLevel'] = df['SeniorityLevel'].apply(lambda x : x.strip())
df['EmployementType'] = df['EmployementType'].apply(lambda x : x.strip())
df['JobFunctions'] = df['JobFunctions'].apply(lambda x : x.strip())
df['Industries'] = df['Industries'].apply(lambda x : x.strip())
df['RelativeTime'] = df['RelativeTime'].apply(lambda x : x.strip())

In [51]:
df

,CompanyNames,JobTitles,CompanyLocation,LinkToJobPostings,JobDescriptions,SeniorityLevel,EmployementType,JobFunctions,Industries,RelativeTime
0,CGI,Power Platform Developer,"Bengaluru, Karnataka, India",https://ca.linkedin.com/company/cgi?trk=public...,"[\n Position Description, [[]], Externa...",Entry level,Full-time,Engineering and Information Technology,IT Services and IT Consulting,3 days ago
1,CodersBrain,POWER PLATFORM DEVELOPER_BHAWNA BANSAL,"Bengaluru, Karnataka, India",https://in.linkedin.com/company/codersbrain?tr...,"[\n Hello,, [[]], Greetings from , [Cod...",Associate,Full-time,Engineering and Information Technology,IT Services and IT Consulting,1 month ago
2,Technovids Consulting Services,Power Apps Developer,"Bengaluru North, Karnataka, India",https://in.linkedin.com/company/technovids-con...,"[\n PowerApp Developer, [[]], Responsib...",Mid-Senior level,Full-time,Engineering and Information Technology,IT Services and IT Consulting,1 week ago
3,Sapwood Ventures Pvt. Ltd.,Power Platform Developer - PowerApps/PowerApps,"Hyderabad, Telangana, India",https://in.linkedin.com/company/sapwood-ventur...,[\n Job opening in one of the reputed c...,Entry level,Full-time,Engineering and Information Technology,"Software Development, Technology, Information ...",1 month ago
4,NuStar Technologies,Power Apps,"Hyderabad, Telangana, India",https://www.linkedin.com/company/nustar-techno...,"[\n, [[Responsibilities]], , [[Manage the Kel...",Mid-Senior level,Full-time,Other,Information Technology & Services,1 week ago
...,...,...,...,...,...,...,...,...,...,...
931,IBM,Supply Chain Analytics Analyst,"Pune, Maharashtra, India",https://www.linkedin.com/company/ibm?trk=publi...,"[\n R00195113, [[]], Engineering | MCA ...",Associate,Full-time,Supply Chain,Chemical Manufacturing,2 weeks ago
932,Cyient,Cloud Data Architect - Machine Learning,"Bengaluru, Karnataka, India",https://in.linkedin.com/company/cyient?trk=pub...,[\n Urgent requirement for Cloud Data A...,Mid-Senior level,Full-time,Engineering and Information Technology,Staffing and Recruiting,1 month ago
933,Accenture in India,Lead Data Scientist GSC’S,"Bengaluru, Karnataka, India",https://in.linkedin.com/company/accentureindia...,"[\n Job Description, [], [], The health...",Not Applicable,Full-time,Engineering and Information Technology,"Banking, Financial Services, and Investment Ba...",23 hours ago
934,Agoda,Microsoft Dynamics 365 ERP Technical Applicati...,"Chennai, Tamil Nadu, India",https://sg.linkedin.com/company/agoda?trk=publ...,"[\n, [About Accenture: ], Accenture is a globa...",Mid-Senior level,Full-time,Information Technology and Engineering,IT Services and IT Consulting,1 week ago


In [54]:
df['RelativeTime'][0]

'3 days ago'

## Save Data to CSV

In [55]:
df.to_csv('scrappedData.csv', mode='a', index=False, header=False) #df.to_csv('scrappedData.csv',index=False)

## Load Data from CSV

In [56]:
df=pd.read_csv("scrappedData.csv")

In [57]:
df

,CompanyNames,JobTitles,CompanyLocation,LinkToJobPostings,JobDescriptions,SeniorityLevel,EmployementType,JobFunctions,Industries,RelativeTime
0,Aspirion,Director of Business Intelligence,"Atlanta, GA",https://www.linkedin.com/company/aspirion-heal...,"<div class=""show-more-less-html__markup show-m...",Director,Full-time,Business Development and Sales,"Technology, Information and Internet",1 day ago
1,UCLA Health,Business Intelligence Developer,"Los Angeles, CA",https://www.linkedin.com/company/ucla-health?t...,"<div class=""show-more-less-html__markup show-m...",Mid-Senior level,Full-time,Information Technology,Hospitals and Health Care,2 weeks ago
2,Paramount+,Business Intelligence Analyst,"New York, NY",https://www.linkedin.com/company/paramount-plu...,"<div class=""show-more-less-html__markup show-m...",Mid-Senior level,Full-time,"Research, Analyst, and Information Technology",Entertainment Providers,2 days ago
3,Bloom Healthcare,Business Intelligence Analyst,"Wheat Ridge, CO",https://www.linkedin.com/company/bloom-healthc...,"<div class=""show-more-less-html__markup show-m...",Mid-Senior level,Full-time,"Research, Analyst, and Information Technology",Internet Publishing,1 month ago
4,First Soft Solutions LLC,Business Intelligence (BI) Senior Developer,"Piscataway, NJ",https://www.linkedin.com/company/first-soft-so...,"<div class=""show-more-less-html__markup show-m...",Entry level,Full-time,Business Development and Sales,IT Services and IT Consulting,3 months ago
...,...,...,...,...,...,...,...,...,...,...
2904,IBM,Supply Chain Analytics Analyst,"Pune, Maharashtra, India",https://www.linkedin.com/company/ibm?trk=publi...,"<div class=""show-more-less-html__markup show-m...",Associate,Full-time,Supply Chain,Chemical Manufacturing,2 weeks ago
2905,Cyient,Cloud Data Architect - Machine Learning,"Bengaluru, Karnataka, India",https://in.linkedin.com/company/cyient?trk=pub...,"<div class=""show-more-less-html__markup show-m...",Mid-Senior level,Full-time,Engineering and Information Technology,Staffing and Recruiting,1 month ago
2906,Accenture in India,Lead Data Scientist GSC’S,"Bengaluru, Karnataka, India",https://in.linkedin.com/company/accentureindia...,"<div class=""show-more-less-html__markup show-m...",Not Applicable,Full-time,Engineering and Information Technology,"Banking, Financial Services, and Investment Ba...",23 hours ago
2907,Agoda,Microsoft Dynamics 365 ERP Technical Applicati...,"Chennai, Tamil Nadu, India",https://sg.linkedin.com/company/agoda?trk=publ...,"<div class=""show-more-less-html__markup show-m...",Mid-Senior level,Full-time,Information Technology and Engineering,IT Services and IT Consulting,1 week ago


## Push Data to SQL

In [58]:
import pyodbc

In [59]:
server = 'DESKTOP-50GJUM7' 
database = 'ResumeProjects' 

### Connect to SQLServer

In [60]:
cnxn = pyodbc.connect("DRIVER={SQL Server};"
                     "Server="+server+";"+
                     "Database="+database+";"+
                      "Trusted_Connection=yes;")
cursor = cnxn.cursor()

### Create a SQL table

In [ ]:
#createTableQuery = "CREATE TABLE [LinkdInData].[JobDetails] ([CompanyNames] varchar(300),[JobTitles] varchar(500),[CompanyLocation] varchar(100),[LinkToJobPostings] varchar(800),[JobDescriptions] TEXT,[SeniorityLevel] varchar(100),[EmployementType] varchar(100),[JobFunctions] varchar(200),[Industries] varchar(200),[RelativeTime] varchar(100))"

In [ ]:
df.columns

In [61]:
#cursor.execute(createTableQuery)
#cnxn.commit()
cursor.execute("TRUNCATE TABLE LinkdInData.JobDetails")
cnxn.commit()

In [62]:
for index, row in df.iterrows():
     cursor.execute("INSERT INTO LinkdInData.JobDetails (CompanyNames,JobTitles,CompanyLocation,LinkToJobPostings,JobDescriptions,SeniorityLevel,EmployementType,JobFunctions,Industries,RelativeTime) values(?,?,?,?,?,?,?,?,?,?)", row.CompanyNames, row.JobTitles, row.CompanyLocation, row.LinkToJobPostings, row.JobDescriptions, row.SeniorityLevel, row.EmployementType, row.JobFunctions, row.Industries,row.RelativeTime)

In [63]:
cnxn.commit()

In [64]:
cursor.close()